<a href="https://colab.research.google.com/github/research-clone/notebook_tutorials/blob/main/Copy_of_Omer_T5_Paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
# !apt-get remove python3
# !apt-get install python3.8
# !ln -sf /usr/bin/python3.8 /usr/bin/python

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3

!python3 --version
!sudo apt install python3-pip

Setting up python3-pkg-resources (45.2.0-1ubuntu0.1) ...
Setting up python3-distutils (3.8.10-0ubuntu1~20.04) ...
Setting up python3-setuptools (45.2.0-1ubuntu0.1) ...
Setting up python3-dev (3.8.2-0ubuntu2) ...
Setting up python3-pip (20.0.2-5ubuntu1.9) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
!pip install -qq "torch==1.4.0" "pytorch_lightning==1.2.6" "transformers==2.9.0" "sentencepiece" "vncorenlp" "rouge-score==0.0.4" "sentence_transformers" "datasets"

     |████████████████████████████████| 753.4 MB 20 kB/s 
     |████████████████████████████████| 233 kB 95.8 MB/s 
     |████████████████████████████████| 635 kB 73.5 MB/s 
     |████████████████████████████████| 1.3 MB 59.5 MB/s 
     |████████████████████████████████| 2.6 MB 56.5 MB/s 
     |████████████████████████████████| 85 kB 5.6 MB/s 
     |████████████████████████████████| 486 kB 57.9 MB/s 
     |████████████████████████████████| 17.3 MB 59.8 MB/s 
     |████████████████████████████████| 5.6 MB 64.4 MB/s 
     |████████████████████████████████| 840 kB 73.7 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 7.5 MB 53.2 MB/s 
     |████████████████████████████████| 772 kB 62.4 MB/s 
     |████████████████████████████████| 880 kB 68.4 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 1.5 MB 56.4 MB/s 
     |████████████████████████████████| 126 kB 79.6 MB/s 
     |█████████████

In [ ]:
!pip install "pytorch_lightning==0.7.5"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Jul  9 00:39:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import gc
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from tqdm import tqdm

from string import punctuation
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import (
    AdamW,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    get_linear_schedule_with_warmup)


from datasets import load_dataset

# from transformers import (

#     T5ForConditionalGeneration,
#     T5Tokenizer,
#     get_linear_schedule_with_warmup
# )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: ignored

## Make dataset

In [ ]:
paraphrase_dataset = load_dataset("humarin/chatgpt-paraphrases")

In [ ]:
# paraphrase_dataset["train"][0]

In [ ]:
# subset_dataset = dataset["train"].select(range(100000))
# Assuming your dataset is stored in a list called 'dataset'
data = []
for example in tqdm(paraphrase_dataset["train"].select(range(100000)), desc='Processing dataset'):
    sentence_1 = example['text']
    paraphrases = eval(example['paraphrases'])  # Convert the string representation to a list
    num_paraphrased = random.randint(1, len(paraphrases))  # Randomly choose the number of paraphrased sentences

    sentence_2 = ' \n- '.join(paraphrases[:num_paraphrased])  # Concatenate the selected paraphrased sentences

    data.append({'sentence_1': sentence_1, 'sentence_2': sentence_2, 'num_paraphrased': num_paraphrased, 'category': example['category']})

# Convert the data list to a DataFrame
df = pd.DataFrame(data)
df.head()

In [ ]:
train_df , val_df = train_test_split(df, test_size=0.05)

In [ ]:
df['num_paraphrased'].value_counts()

In [ ]:
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_len=50):

        self.source_column = "sentence_1"
        self.target_column = "sentence_2"
        self.num_paraphrased_column = "num_paraphrased"
        self.data = dataframe

        self.source_input = self.data[self.source_column].to_list()
        self.source_output = self.data[self.target_column].to_list()
        self.num_paraphrased = self.data[self.num_paraphrased_column].to_list()

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in tqdm(range(len(self.data))):
            input_, target, num_para = self.source_input[idx], self.source_output[idx], self.num_paraphrased[idx]

            input_ = f"paraphrase {num_para} sentences: "+ input_ + ' </s>'
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, padding="max_length", return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, padding="max_length", return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
def get_dataset(tokenizer, dataframe):
  return ParaphraseDataset(tokenizer=tokenizer, dataframe=dataframe)

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hyperparams):
        super(T5FineTuner, self).__init__()
        self.hyperparams = hyperparams


        self.model = AutoModelForSeq2SeqLM.from_pretrained(hyperparams.model_name_or_path).to()
        self.tokenizer = AutoTokenizer.from_pretrained(hyperparams.tokenizer_name_or_path)

        self.train_dataset = get_dataset(tokenizer=self.tokenizer,dataframe=train_df)
        print("Loaded train dataset")
        self.val_dataset = get_dataset(tokenizer=self.tokenizer, dataframe=val_df)
        print("Loaded val dataset")

    def is_logger(self):
        return self.trainer.global_rank <= 0

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        # print("training step")
        return loss

    def training_step(self, batch, batch_idx):

        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        gc.collect()
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        self.eval()
        with torch.no_grad():
          loss = self._step(batch)
          return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):

        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hyperparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hyperparams.learning_rate, eps=self.hyperparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self,
                    epoch=None,
                    batch_idx=None,
                    optimizer=None,
                    optimizer_idx=None,
                    optimizer_closure=None,
                    on_tpu=None,
                    using_native_amp=None,
                    using_lbfgs=None):
        # print("optimizer_step")

        optimizer.step(closure=optimizer_closure) # remove 'closure=optimizer_closure' here
        optimizer.zero_grad()
        self.lr_scheduler.step()



    def get_tqdm_dict(self):
        print("get tqdm")
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        print("load_train")
        dataloader = DataLoader(self.train_dataset, batch_size=self.hyperparams.train_batch_size, drop_last=True, shuffle=True,num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hyperparams.train_batch_size * max(1, self.hyperparams.n_gpu)))
                // self.hyperparams.gradient_accumulation_steps
                * float(self.hyperparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hyperparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.hyperparams.eval_batch_size, num_workers=4)

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hyperparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
gc.collect()

In [ ]:
args_dict = dict(
    output_dir="", # path to save the checkpoints
    model_name_or_path='google/flan-t5-base',
    tokenizer_name_or_path='google/flan-t5-base',
    max_seq_length=256,
    learning_rate=3e-5,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

In [ ]:
if not os.path.exists('t5_paraphrase'):
    os.makedirs('t5_paraphrase')

args_dict.update({'output_dir': 't5_paraphrase'})
args = argparse.Namespace(**args_dict)

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=5
)

model = T5FineTuner(args)

In [ ]:
model.summarize()
# model

In [ ]:
gc.collect()

In [ ]:
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    gradient_clip_val=args.max_grad_norm
    # checkpoint_callback=checkpoint_callback,
    # callbacks=[LoggingCallback()],
)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

In [ ]:
save_cp = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/checkpoint"
model.model.save_pretrained(save_cp+'/t5_paraphrase_v2')

In [ ]:
save_cp = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/checkpoint"
infer_model = AutoModelForSeq2SeqLM.from_pretrained(save_cp+'/t5_paraphrase')#.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [ ]:
multi_sample = [
    # "Hiện trường có phương thức tuyển sinh nào ?",
                "Trường đại học Quốc Gia đang xét tuyển theo hình thức nào ?",
                # "Trường ĐHQG đang xét tuyển như thế nào ?",
                # "Có những phương pháp xét tuyển nào ?",
                # "Nhà trường có những phương thức xét tuyển nào ?",
                # "Phương thức tuyển sinh của trường Đại học Quốc Gia Hà Nội ?"
                ]

In [ ]:
multi_sample = [
    "Chỉ cho tôi những quán ăn ngon ở Hà Nội",
    "Có quán ăn nào ngon ở Hà Nội"
]

In [ ]:
final_outputs = set()
for sentence in multi_sample:
  text =  "paraphrase: " + sentence + " </s>"
  max_len = 256

  encoding = tokenizer.encode_plus(text,padding=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')

  beam_outputs = infer_model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=30,
      top_k=70,
      early_stopping=True,
      num_return_sequences=1,
      num_beams=3,
      temperature=1.5
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.add(sent)

print("\nOriginal sentence: \n")
print("\n".join(multi_sample))
print("\n")
print("Paraphrased sentences: ")
for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

In [ ]:
class ParaPhrasing:
    def __init__(self,model_path):
        self.model_name = model_path
        self.torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name).to(self.torch_device)

    def paraphrases(self, input_text, num_return_sequences=10, num_beams=10,top_k=12,temperature=1.5,unique_ngram=2):
        """
        generates variations for
        a given sentence/text
        :param input_text: sentence or text
        :param num_return_sequences: Number of variations to be returned
        :param num_beams: Number of beams for beam search. 1 means no beam search
        :return: list of variations of the input text
        """
        if isinstance(input_text, str):
            input_text = ["paraphrase: " + input_text + " </s>"]
        else:
            input_text = ["".join(["paraphrase: " + text + " </s>"]) for text in input_text]
        batch = self.tokenizer.prepare_seq2seq_batch(input_text, truncation=True, padding='longest',
                                                     max_length=60, return_tensors="pt").to(
            self.torch_device)

        translated = self.model.generate(**batch,
                                         do_sample=True,
                                         early_stopping=True,
                                         max_length=60,
                                         num_beams=num_beams,
                                         num_return_sequences=num_return_sequences,
                                         temperature=temperature,
                                         top_k=top_k,
                                         no_repeat_ngram_size=unique_ngram)
        tgt_text = self.tokenizer.batch_decode(translated, skip_special_tokens=True)
        return list(set(tgt_text))

In [ ]:
pg = ParaPhrasing("/content/drive/MyDrive/Colab Notebooks/Paraphrasing/checkpoint/t5_paraphrase_v2")

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

88

In [ ]:
pg.paraphrases(["Thời tiết hôm nay như thế nào ?"],num_return_sequences=30,num_beams=20,top_k=20,temperature=2.0,unique_ngram=2)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['Ngày hôm nay, thời tiết như thế nào? ',
 'Bạn nghĩ gì về thời tiết hôm nay? ',
 'Hôm nay, thời tiết như thế nào? ',
 'Ngày hôm nay thời tiết như thế nào? ',
 'Thời tiết hôm nay như thế nào? ']

In [ ]:
sample = [
          "Thời tiết ở Hà Nội rất nắng nóng",
          "Hôm nay bạn có rảnh không, đi chơi với mình nhé",
          "thời tiết hôm nay như thế nào ?",
          "làm thế nào để đẹp trai hơn ?",
          "làm thế nào để trở thành lập trình viên ?",
          "mức lương của lập trình viên trung bình là bao nhiêu ?",
          "làm thế nào để trở thành một nhà lãnh đạo giỏi ?",
          "Ai là tổng thống Mỹ hiện tại ?"]

In [ ]:
sample = ["Tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến sống ở Vienna (Áo)."]

In [ ]:
for sentence in sample:
  text =  "paraphrase: " + sentence + " </s>"
  # paraphrase: "
  max_len = 256

  encoding = tokenizer.encode_plus(text,padding=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = infer_model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=70,
      top_k=15,
      # top_p=0.5,
      early_stopping=True,
      num_return_sequences=15,

  )


  print ("\nOriginal sentence: ")
  print (sentence)
  print ("\n")
  print ("Paraphrased sentences: ")
  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  for i, final_output in enumerate(final_outputs):
      print("{}: {}".format(i, final_output))


Original sentence: 
Tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến sống ở Vienna (Áo).


Paraphrased sentences: 
0: Tại Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến Vienna (Áo). 
1: Anh đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo). 
2: Tôi có việc làm ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến sống ở Vienna (Áo). 
3: Tôi hiện đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo). 
4: Tôi đang làm việc ở Stuttgart và Berlin (Đức) trong tuần tới sẽ chuyển đến Vienna (Áo). 
5: Tôi đang làm việc ở Stuttgart và Berlin (Đức), sau đó tôi sẽ chuyển đến Vienna (Áo). 
6: Tôi hiện đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo). 
7: Tuần qua tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ sống ở Vienna (Áo). 
8: Tôi đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần sau đó tôi sẽ chuyển đến Vienna (Áo). 
9: Ở Stuttgart và Berlin (Đức), tuần t

# Evaluation

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from tqdm import tqdm
from vncorenlp import VnCoreNLP
import itertools

from transformers import (
    AdamW,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    get_linear_schedule_with_warmup)


In [ ]:
save_cp = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/checkpoint"
infer_model = AutoModelForSeq2SeqLM.from_pretrained(save_cp+'/t5_paraphrase_v2').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")

NameError: ignored

In [ ]:
sbert_model = SentenceTransformer('vinai/phobert-base').to('cuda')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/vinai_phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/vinai_phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word 

In [ ]:
annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/Datasets/ViMS (300ct)/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g')

In [ ]:
save_path = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/data"
with open(save_path+"/test.txt") as f:
  test_data = [line.strip() for line in f]

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def encoded_cosine_similarity(bert, output, y):
    y = bert.encode([y])
    output = bert.encode([output])
    score = cosine_similarity(y, output).squeeze(0)
    return float(score)
def calculate_best_candidate(bert, outputs, y):
    best_rs = 1.0
    best_cs = 0.0
    for output in outputs:
        cs = encoded_cosine_similarity(bert, output, y)
        rs = scorer.score(output, y)['rougeL'].fmeasure
        if rs >= 0.9:
            continue
        elif ((cs + 0.05 >= best_cs) and (rs <  best_rs)) or (cs >= best_cs + 0.045):
            best_cs = cs
            best_rs = rs
            best_output = output
        elif best_rs == 1.0 and best_cs == 0.0:
            best_cs = cs
            best_rs = rs
    return best_rs, best_cs

In [ ]:
def tokenize_text(text):
  text = annotator.tokenize(text)
  return ' '.join(list(itertools.chain.from_iterable(text)))

In [ ]:
tokenize_text("Tôi đang học tại đại học NEU")

'Tôi đang học tại đại_học NEU'

In [ ]:
def evaluate(test_sentences,sbert_model,infer_model):
  best_rouge = []
  best_cosim = []


  for sentence in tqdm(test_sentences):
    text =  "paraphrase: " + sentence + " </s>"
    max_len = 256
    encoding = tokenizer.encode_plus(text,padding=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')

    beam_outputs = infer_model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=70,
        top_k=15,
        early_stopping=True,
        num_return_sequences=5,

    )
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(tokenize_text(sent))
    tokenized_input = tokenize_text(sentence)

    rs, cs = calculate_best_candidate(sbert_model,final_outputs,tokenized_input)
    best_rouge.append(rs)
    best_cosim.append(cs)
  return sum(best_rouge) / len(best_rouge), sum(best_cosim) / len(best_cosim)

In [ ]:
sample = ["Đại học Kinh Tế Quốc Dân là ngôi trường hàng đầu đào tạo về lĩnh vực kinh tế",
          "Thời tiết ở Hà Nội rất nắng nóng",
          "Hôm nay bạn có rảnh không, đi chơi với mình nhé",
          "thời tiết hôm nay như thế nào ?",
          "làm thế nào để đẹp trai hơn ?",
          "làm thế nào để trở thành lập trình viên ?",
          "mức lương của lập trình viên trung bình là bao nhiêu ?",
          "làm thế nào để trở thành một nhà lãnh đạo giỏi ?",
          "Ai là tổng thống Mỹ hiện tại ?"]

In [ ]:
avg_rouge, avg_cosim = evaluate(test_data,sbert_model,infer_model)

100%|██████████| 3726/3726 [46:58<00:00,  1.32it/s]


In [ ]:
print(f"Total sentence:                  {len(test_data)}")
print(f"Average Rouge-L socre:           {avg_rouge} (lower is better)")
print(f"Average Cosine Similarity socre: {avg_cosim} (higher is better)")

Total sentence:                  3726
Average Rouge-L socre:           0.7389676343142427 (lower is better)
Average Cosine Similarity socre: 0.7949705829686908 (higher is better)


## Test perplexity socre of an sentences

In [ ]:
!pip install -qq transformers

     |████████████████████████████████| 4.4 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 6.6 MB 36.7 MB/s 
     |████████████████████████████████| 101 kB 9.6 MB/s 


In [ ]:
import torch
import sys
import numpy as np

from transformers import AutoTokenizer, GPTNeoForCausalLM
# Load pre-trained model (weights)
with torch.no_grad():
        model = GPTNeoForCausalLM.from_pretrained('VietAI/gpt-neo-1.3B-vietnamese-news').to('cuda')
        model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = AutoTokenizer.from_pretrained("VietAI/gpt-neo-1.3B-vietnamese-news")



Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

In [ ]:
import torch
import sys
import numpy as np

from transformers import AutoTokenizer, AutoModelForMaskedLM
# Load pre-trained model (weights)
with torch.no_grad():
        pb_model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base").to('cuda')
        pb_model.eval()
# Load pre-trained model tokenizer (vocabulary)
pb_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")




Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def ppl_score(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input]).to('cuda')
    loss=model(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.cpu().detach().numpy())

In [ ]:
import pandas as pd
texts = ["Tôi đang học tại đại học NEU",
"Anh đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo). ",
"Tôi có việc làm ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến sống ở Vienna (Áo).",
"Tôi hiện đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức) trong tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức), sau đó tôi sẽ chuyển đến Vienna (Áo)." ,
"Tôi hiện đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tuần qua tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ sống ở Vienna (Áo). ",
"Tôi đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần sau đó tôi sẽ chuyển đến Vienna (Áo).",
"Ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến sống ở Vienna (Áo). ",
"Tôi đang sống ở Berlin (Đức), tuần tới tôi sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức) và tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc tại Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc tại Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo)." ]


scores = [str(ppl_score(line.strip())) for line in texts]
pd.DataFrame({
    "text": texts,
    "PPL Score": scores
})

,text,PPL Score
0,Tôi đang học tại đại học NEU,94.95172
1,"Anh đang làm việc ở Stuttgart và Berlin (Đức),...",40.485752
2,"Tôi có việc làm ở Stuttgart và Berlin (Đức), t...",32.389435
3,Tôi hiện đang làm việc ở Stuttgart và Berlin (...,31.129408
4,Tôi đang làm việc ở Stuttgart và Berlin (Đức) ...,40.31986
5,"Tôi đang làm việc ở Stuttgart và Berlin (Đức),...",26.835344
6,Tôi hiện đang làm việc ở Stuttgart (Đức) và Be...,21.580923
7,Tuần qua tôi đang làm việc ở Stuttgart và Berl...,32.31259
8,Tôi đang làm việc ở Stuttgart (Đức) và Berlin ...,22.086723
9,"Ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ c...",31.758379


In [ ]:
import pandas as pd
texts = ["Tại Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến đại học Vienna (Áo).",
"Anh đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo). ",
"Tôi có việc làm ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến sống ở Vienna (Áo).",
"Tôi hiện đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức) trong tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức), sau đó tôi sẽ chuyển đến Vienna (Áo)." ,
"Tôi hiện đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tuần qua tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ sống ở Vienna (Áo). ",
"Tôi đang làm việc ở Stuttgart (Đức) và Berlin (Đức), tuần sau đó tôi sẽ chuyển đến Vienna (Áo).",
"Ở Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến sống ở Vienna (Áo). ",
"Tôi đang sống ở Berlin (Đức), tuần tới tôi sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức) và tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc tại Stuttgart và Berlin (Đức), tuần tới tôi sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc tại Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo).",
"Tôi đang làm việc ở Stuttgart và Berlin (Đức), tuần tới sẽ chuyển đến Vienna (Áo)." ]

texts = [tokenize_text(line.strip()) for line in texts]
scores = [str(ppl_score(line)) for line in texts]
pd.DataFrame({
    "text": texts,
    "PPL Score": scores
})

,text,PPL Score
0,"Tại Stuttgart và Berlin ( Đức ) , tuần tới tôi...",7.780117
1,Anh đang làm_việc ở Stuttgart và Berlin ( Đức ...,7.4261956
2,Tôi có việc_làm ở Stuttgart và Berlin ( Đức ) ...,4.528194
3,Tôi hiện đang làm_việc ở Stuttgart và Berlin (...,5.486202
4,Tôi đang làm_việc ở Stuttgart và Berlin ( Đức ...,9.472947
5,Tôi đang làm_việc ở Stuttgart và Berlin ( Đức ...,5.5940213
6,Tôi hiện đang làm_việc ở Stuttgart ( Đức ) và ...,5.0026608
7,Tuần qua tôi đang làm_việc ở Stuttgart và Berl...,5.4873037
8,Tôi đang làm_việc ở Stuttgart ( Đức ) và Berli...,4.430839
9,"Ở Stuttgart và Berlin ( Đức ) , tuần tới tôi s...",4.2954974


In [ ]:
!pip install -qq sentence_transformers

In [ ]:
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2').to('cuda')

In [ ]:
!pip install -qq "vncorenlp" "rouge_score" "sentence_transformers"

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2').to('cuda')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]